In [ ]:
import numpy as np
import pandas as pd
import datetime
import os
import gc

In [ ]:
dfLinkTs = None

for f in os.listdir('../input'):
    if not 'JT Count Data' in f:
        continue
    
    print('Processing "%s"...' % f)
    
    dfLinks = pd.read_csv(
        '../input/' + f,
        names=['systemCodeIdentifier', 'time', 'platesIn', 'platesOut'],
        dtype={
            'systemCodeIdentifier': str,
            'time': str,
            'platesIn': np.float64,
            'platesOut': np.float64
        },
        thousands=',',
        low_memory=True
    )
    dfLinks['time'] = dfLinks['time'].apply(lambda t: datetime.datetime.strptime(t, '%d/%m/%Y %H:%M:%S'))
    dfLinks = dfLinks.pivot(
        index='time',
        columns='systemCodeIdentifier'
    )
    dfLinks = dfLinks.resample('240s').sum()
    dfLinks.columns = ['.'.join(reversed(col)).strip() for col in dfLinks.columns.values]
    
    print('  Loaded %u timeseries entries.' % len(dfLinks.index))
    
    if dfLinkTs is None:
        dfLinkTs = dfLinks
    else:
        dfLinkTs = pd.concat([dfLinkTs, dfLinks])
        
    gc.collect()

In [ ]:
dfLinkTs

In [ ]:
dfLinkTs.to_pickle('../cache/baseline-traffic-volumes-pd.pkl')

In [ ]:
list(dfLinkTs.columns)